In [1]:
import torch
import numpy as np
import ase.io
import tqdm

import sys
sys.path.append('../code/')
from code_pytorch import *
from utilities import *

from nice.clebsch_gordan import ClebschGordan
from nice.rascal_coefficients import process_structures
from nice.utilities import get_all_species, get_spherical_expansion

# Data preparation

In [2]:
LAMBDA_MAX = 4

HYPERS = {
    'interaction_cutoff': 6.3,
    'max_radial': 4,
    'max_angular': LAMBDA_MAX,
    'gaussian_sigma_type': 'Constant',
    'gaussian_sigma_constant': 0.3,
    'cutoff_smooth_width': 0.3,
    'radial_basis': 'GTO'
}
DEVICE = 'cuda' #or 'cpu'
clebsch = ClebschGordan(LAMBDA_MAX)

In [3]:
structures = process_structures(ase.io.read('methane.extxyz', index='0:10'))
all_species = get_all_species(structures)
coefficients_np = get_spherical_expansion(structures, HYPERS,
                                             all_species, split_by_central_specie=False, 
                                         show_progress = False)
coefficients = {}
for l in range(LAMBDA_MAX + 1):
    coefficients[l] = torch.from_numpy(coefficients_np[:, :, l, :(2 * l + 1)]).type(torch.get_default_dtype()).to(DEVICE)
    print(l, coefficients[l].shape)

FileNotFoundError: [Errno 2] No such file or directory: 'methane.extxyz'

# invariants calculation

### ps invariants

In [ ]:
block = ClebschCombining(clebsch.precomputed_, 0).to(DEVICE)
ps = block(coefficients, coefficients)[0].squeeze()
print(ps.shape)

### ps covariants

In [ ]:
block = ClebschCombining(clebsch.precomputed_, LAMBDA_MAX).to(DEVICE)
ps_covariants = block(coefficients, coefficients)
for key in ps_covariants.keys():
    print(key, ps_covariants[key].shape)

### bs invariants

In [ ]:
first = ClebschCombining(clebsch.precomputed_, LAMBDA_MAX).to(DEVICE)
second = ClebschCombining(clebsch.precomputed_, 0).to(DEVICE)

bs_invariants = first(second(coefficients, coefficients), coefficients)[0].squeeze()
print(bs_invariants.shape)

### bs covariants

In [ ]:
first = ClebschCombining(clebsch.precomputed_, LAMBDA_MAX).to(DEVICE)
second = ClebschCombining(clebsch.precomputed_, LAMBDA_MAX).to(DEVICE)

bs_invariants = first(second(coefficients, coefficients), coefficients)
for key in bs_invariants.keys():
    print(key, bs_invariants[key].shape)

# proper shaped code

In [ ]:
class Powerspectrum(torch.nn.Module):
    def __init__(self, clebsch):
        super(Powerspectrum, self).__init__()
        self.first = ClebschCombining(clebsch, 0)       
            
    def forward(self, X):
        ps_invariants = self.first(X, X)[0].squeeze()       
        return ps_invariants
    
class Bispectrum(torch.nn.Module):
    def __init__(self, clebsch, lambda_max):
        super(Bispectrum, self).__init__()
        self.first = ClebschCombining(clebsch, lambda_max)
        self.second = ClebschCombining(clebsch, 0)
            
    def forward(self, X):
        ps_covariants = self.first(X, X)
        bs_invariants = self.second(ps_covariants, X)[0].squeeze()
        return bs_invariants
    
class Nspectrum(torch.nn.Module):
    def __init__(self, order, clebsch, lambda_max):
        super(Nspectrum, self).__init__()
        self.combinings = torch.nn.ModuleList([ClebschCombining(clebsch, lambda_max) for _ in range(order - 1)] + \
                                           [ClebschCombining(clebsch, 0)])
        
    def forward(self, X):
        current = X
        for combining in self.combinings:
            current = combining(current, X)
        return current[0].squeeze()

In [ ]:
ps_model =  Powerspectrum(clebsch.precomputed_).to(DEVICE)
print("powerspectrum: ",ps_model(coefficients).shape)
bs_model = Bispectrum(clebsch.precomputed_, LAMBDA_MAX).to(DEVICE)
print("bispectrum: ", bs_model(coefficients).shape)
ts_model = Nspectrum(3, clebsch.precomputed_, LAMBDA_MAX).to(DEVICE) #try other values as order
coef_small = {key : value[0:3] for key, value in coefficients.items()} #trispectrum is quite huge
print("trispectrum: ", ts_model(coef_small).shape)

In [7]:
a = np.zeros(100)
print(type(a))
print(type(a) == dict)
b = {'asdf' : 234}
print(type(b) == dict)

<class 'numpy.ndarray'>
False
True
